In [2]:
# Importation des librairies
from my_function import ( 
    sel_classif_pixel,
    report_from_dict_to_df,
    supprimer_dossier_non_vide,
    rasterization,
    id_construction,
    stratified_grouped_validation,
    save_classif)
import os
import matplotlib.pyplot as plt
from osgeo import gdal
import geopandas as gpd
import numpy as np
import pandas as pd
import sys
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import (StratifiedKFold, StratifiedGroupKFold)
from sklearn.metrics import (confusion_matrix, classification_report,
    accuracy_score)
sys.path.append('/home/onyxia/work/libsigma')
import read_and_write as rw
import classification as cla
import plots

# Création du dossier de sauvegarde temporaire et paramètres
racine = "/home/onyxia/work"
output_dir = os.path.join(racine,"output_classif")
os.makedirs(output_dir, exist_ok=True) # Création d'un dossier output temporaire

path_sample = os.path.join(racine,"results/data/sample/Sample_BD_foret_T31TCJ.shp")
path_sample_px = os.path.join(output_dir,"sample_classif_px.shp")

sample_rasterized = os.path.join(output_dir,"rasterized_sample.tif")
path_image_3bands = os.path.join(racine,"results/data/img_pretraitees/Serie_temp_S2_3_band.tif")
path_image_allbands = os.path.join(racine,"results/data/img_pretraitees/Serie_temp_S2_allbands.tif")

path_sample_px_centroid = os.path.join(output_dir,"sample_px_centroid.shp")
path_sample_px_id = os.path.join(output_dir,"sample_px_id.shp")
path_rasterized_sample_id = os.path.join(output_dir, "rasterized_sample_id.tif")

out_classif = os.path.join(racine, "results/data/classif/carte_essences_echelle_pixel.tif")

# Save d'un vecteur echantillons avec que les données pour classif pixel
sample = gpd.read_file(path_sample)
sample_px = sel_classif_pixel(sample[['Code',"geometry"]])
sample_px.to_file(path_sample_px)

# Rasterisation des échantillons pour la classification pixel
rasterization(
    in_vector=path_sample_px,
    out_image=sample_rasterized,
    field_name='Code',
    ref_image=path_image_allbands,
    dtype='Byte'
)

# Construction du raster id pour la méthode stratifiée groupée
id_construction(sample_px,path_sample_px_id)
rasterization(
    in_vector=path_sample_px_id,
    out_image=path_rasterized_sample_id,
    field_name="id",
    ref_image=path_image_allbands
)


0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
# Entrainement du modèle et validation stratifiée groupée
id_filename = path_rasterized_sample_id
nb_iter = 30
nb_folds = 5
rfc, list_cm, list_accuracy, list_report, Y_predict =\
stratified_grouped_validation(
    nb_iter=nb_iter,
    nb_folds=nb_folds,
    sample_filename=sample_rasterized,
    image_filename=path_image_allbands,
    id_filename=id_filename
)

Début de la 1 itération
{'11': {'precision': 0.8605382733310032, 'recall': 0.23153242112192599, 'f1-score': 0.3648893993849346, 'support': 21267.0}, '12': {'precision': 0.938241748398943, 'recall': 0.9982845706661584, 'f1-score': 0.9673323329100774, 'support': 419720.0}, '13': {'precision': 0.7178217821782178, 'recall': 0.09631351710395218, 'f1-score': 0.1698389458272328, 'support': 3011.0}, '14': {'precision': 0.5341047503045067, 'recall': 0.16313244047619047, 'f1-score': 0.249928754630949, 'support': 5376.0}, '22': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 259.0}, '23': {'precision': 0.3507853403141361, 'recall': 0.15635939323220538, 'f1-score': 0.21630347054075869, 'support': 1714.0}, '24': {'precision': 0.8960830479452054, 'recall': 0.6421997238840313, 'f1-score': 0.748190510231436, 'support': 13038.0}, '25': {'precision': 0.6070816676185037, 'recall': 0.6458080194410692, 'f1-score': 0.6258463350014719, 'support': 1646.0}, 'micro avg': {'precision': 0.9323802130

In [ ]:
# Stratégie d'évitement : supprimer les tableaux qui n'ont pas toutes les classes
list_report_2 = []
list_cm_2 = []
for report in list_report:
    nb_report = len(report.keys())
    if nb_report == 9 :
        list_report_2.append(report)
print(len(list_report_2))
for cm in list_cm:
    nb_cm = len(cm)
    if nb_cm == 9:
        list_cm_2.append(cm)
print(len(list_cm_2))

In [ ]:
# Stratégie de remplissage 0
list_head = ['11', '12', '13', '14', '21', '22', '23', '24', '25']
l_manque = []
for report in list_report:
    for head in list_head:
        if head not in report.keys():
            report[head] = [0,0,0]
            report.columns = report.columns.sort_values()
list_report_2 = list_report.copy()

In [ ]:
## Plots
suffix = '_CV{}folds_stratified_group_x{}times'.format(nb_folds, nb_iter)
out_figs_dir = os.path.join(racine,"results/figure")
out_matrix = os.path.join(out_figs_dir, 'matrice{}.png'.format(suffix))
out_qualite = os.path.join(out_figs_dir, 'qualites{}.png'.format(suffix))

# compute mean of cm
array_cm = np.array(list_cm_2)
mean_cm = array_cm.mean(axis=0)

# compute mean and std of overall accuracy
array_accuracy = np.array(list_accuracy)
mean_accuracy = array_accuracy.mean()
std_accuracy = array_accuracy.std()

# compute mean and std of classification report
array_report = np.array(list_report_2)
mean_report = array_report.mean(axis=0)
std_report = array_report.std(axis=0)
a_report = list_report_2[0]
mean_df_report = pd.DataFrame(mean_report, index=a_report.index,
                              columns=a_report.columns)
std_df_report = pd.DataFrame(std_report, index=a_report.index,
                             columns=a_report.columns)

# Display confusion matrix
plots.plot_cm(mean_cm, np.unique(Y_predict))
plt.savefig(out_matrix, bbox_inches='tight')

# Display class metrics
fig, ax = plt.subplots(figsize=(10, 7))
ax = mean_df_report.T.plot.bar(ax=ax, yerr=std_df_report.T, zorder=2)
ax.set_ylim(0.5, 1)
_ = ax.text(1.5, 0.95, 'OA : {:.2f} +- {:.2f}'.format(mean_accuracy,
                                                      std_accuracy),
            fontsize=14)
ax.set_title('Class quality estimation')

# custom : cuteness
# background color
ax.set_facecolor('ivory')
# labels
x_label = ax.get_xlabel()
ax.set_xlabel(x_label, fontdict={'fontname': 'Sawasdee'}, fontsize=14)
y_label = ax.get_ylabel()
ax.set_ylabel(y_label, fontdict={'fontname': 'Sawasdee'}, fontsize=14)
# borders
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.tick_params(axis='x', colors='darkslategrey', labelsize=14)
ax.tick_params(axis='y', colors='darkslategrey', labelsize=14)
# grid
ax.minorticks_on()
ax.yaxis.grid(which='major', color='darkgoldenrod', linestyle='--',
              linewidth=0.5, zorder=1)
ax.yaxis.grid(which='minor', color='darkgoldenrod', linestyle='-.',
              linewidth=0.3, zorder=1)
plt.savefig(out_qualite, bbox_inches='tight')